In [1]:
!pip install -q --upgrade pip
!pip install jaxlib==0.4.4
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 41.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexe

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [2]:
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax

from utils import *

In [87]:
# define a toy 4x4 graph
# 0----3
# |    |
# 1----2


edge_list = np.array([[0,1],
                      [0,3],
                      [1,0],
                      [1,2],
                      [2,1],
                      [2,3],
                      [3,2],
                      [3,0]])

A = to_dense(np.transpose(edge_list), 4)
print(A)

node_features = np.array([[1,2,3,4,5,6,7],
                          [10,20,30,40,50,60,70],
                          [100,200,300,400,500,600,700],
                          [1000,2000,3000,4000,5000,6000,7000]])

node_features = node_features * 1.0

node_features


[[0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 0.]]


Array([[1.e+00, 2.e+00, 3.e+00, 4.e+00, 5.e+00, 6.e+00, 7.e+00],
       [1.e+01, 2.e+01, 3.e+01, 4.e+01, 5.e+01, 6.e+01, 7.e+01],
       [1.e+02, 2.e+02, 3.e+02, 4.e+02, 5.e+02, 6.e+02, 7.e+02],
       [1.e+03, 2.e+03, 3.e+03, 4.e+03, 5.e+03, 6.e+03, 7.e+03]],      dtype=float32, weak_type=True)

In [88]:
 expected = A @ node_features
 expected

Array([[1010., 2020., 3030., 4040., 5050., 6060., 7070.],
       [ 101.,  202.,  303.,  404.,  505.,  606.,  707.],
       [1010., 2020., 3030., 4040., 5050., 6060., 7070.],
       [ 101.,  202.,  303.,  404.,  505.,  606.,  707.]], dtype=float32)

Simple example much like presented here: https://neural-tangents.readthedocs.io/en/latest/_autosummary/neural_tangents.stax.Aggregate.html#neural_tangents.stax.Aggregate

The problem with this is, that we can not run convolution layer on this input shape?! We need to reshape the node features such that they have a channel dimension

In [89]:
# reshape
# the node feature must be a n x 1 dimensional array, with a channel dim

# add a batch dimension
x = np.expand_dims(node_features, 0)

pattern = np.expand_dims(A, 0)

print(pattern.shape)
print(x.shape)

(1, 4, 4)
(1, 4, 7)


In [90]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=1, batch_axis=0, channel_axis=2)
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [91]:
out = apply_fn(params, x, pattern=pattern)
out == expected

Array([[[ True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True]]], dtype=bool)

Now with a channel dimension

In [92]:
# reshape
# the node feature must be a n x 1 dimensional array, with a channel dim

# add a batch dimension
x = np.expand_dims(node_features, 0)

# add a dim to make the node features a n x 1 dimensional array
# with a channel dim
x = np.expand_dims(x, 2)

pattern = np.expand_dims(A, 0)
pattern = np.expand_dims(pattern, 2)
pattern = np.expand_dims(pattern, 4)

print(pattern.shape)
print(x.shape)

(1, 4, 1, 4, 1)
(1, 4, 1, 7)


In [153]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3),
    stax.Conv(1, (1,1), parameterization="standard")
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [94]:
# set all weights to 1 for the convolution
params = ((), (np.ones(params[1][0].shape), None))

In [95]:
out = apply_fn(params, x, pattern=pattern)
np.squeeze(out).shape

(4, 2)

In [96]:
# the convolution weights are all 1 => the output of the convlution is the sum of the output of the agg layer
out = apply_fn(params, x, pattern=pattern)
print(np.squeeze(out))
np.sum(expected, axis=1)

[[28280. 28280.]
 [ 2828.  2828.]
 [28280. 28280.]
 [ 2828.  2828.]]


Array([28280.,  2828., 28280.,  2828.], dtype=float32)

Test the kernel function

In [102]:
layer_wide = 16
# define a grap convolution network
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3, implementation="DENSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="DENSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.GlobalSumPool(),
    stax.Dense(1),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

a = apply_fn(params, x, pattern=pattern)
k = kernel_fn(x, x, ('ntk', "nngp"), pattern=(pattern, pattern))

print(k.ntk.shape)
print(k.nngp.shape)
print(x.shape)
print(pattern.shape)

(1, 1)
(1, 1)
(1, 4, 1, 7)
(1, 4, 1, 4, 1)


Now with spars pattern
The shape of the pattern is: batch x #edges x 1 x 2

In [143]:
# reshape
# the node feature must be a n x 1 dimensional array, with a channel dim

# add a batch dimension
x = np.expand_dims(node_features, 0)

# add a dim to make the node features a n x 1 dimensional array
# with a channel dim
x = np.expand_dims(x, 2)

pattern = np.expand_dims(edge_list, 0)
pattern = np.expand_dims(pattern, 2)

print(pattern.shape)
print(x.shape)

(1, 8, 1, 2)
(1, 4, 1, 7)


In [144]:
# need to add a copy at axis 2 to make the argegation over axis (1,2) work
# need to aggreagte over axis (1,2) to keep the shape of the kernel stable in the aggregation layer
pattern = np.append(pattern, pattern, axis=2)
print(pattern.shape)

(1, 8, 2, 2)


In [145]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE")
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [146]:
out = apply_fn(params, x, pattern=pattern)
print(out.shape)
np.squeeze(out) == expected

(1, 4, 1, 7)


Array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True]], dtype=bool)

Add conv layer

In [155]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.Conv(1, (1,1), parameterization="standard")
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [156]:
# set all weights to 1 for the convolution
params = ((), (np.ones(params[1][0].shape), None))

In [158]:
# the convolution weights are all 1 => the output of the convlution is the sum of the output of the agg layer
out = apply_fn(params, x, pattern=pattern)
print(np.squeeze(out))
np.sum(expected, axis=1)

[28280.  2828. 28280.  2828.]


Array([28280.,  2828., 28280.,  2828.], dtype=float32)

Test the kernel function:

In [136]:
pattern = np.append(pattern, pattern, axis=2)

In [154]:
layer_wide = 16
# define a grap convolution network
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    # stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    # stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    # stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    # stax.GlobalSumPool(),
    # stax.Dense(1),
)


print(x.shape)
print(pattern.shape)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

a = apply_fn(params, x, pattern=pattern)
k = kernel_fn(x, x, ('ntk', "nngp"), pattern=(pattern, pattern))

print(k.ntk.shape)
print(k.nngp.shape)

(1, 4, 1, 7)
(1, 8, 2, 2)
(1, 1, 1, 1, 4, 4)
(1, 1, 1, 1, 4, 4)
